<a href="https://colab.research.google.com/github/ammn23/recsystem/blob/main/recipe_recsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
recipes=pd.read_csv('/content/drive/MyDrive/recipes_w_search_terms.csv')

In [ ]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}"
2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg..."
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}"
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'}


In [ ]:
recipes['diabetic'] = recipes['tags'].apply(lambda x: 1 if 'diabetic' in x else 0)

In [ ]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",0
2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg...",0
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}",0
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},0


In [ ]:
recipes['id']=recipes['id'].astype(str)

In [ ]:
recipes.dropna(subset=['name'], inplace=True)

In [ ]:
recipes.dropna(subset=['description'], inplace=True)

In [ ]:
print('duplicate rows',recipes.duplicated().sum())

duplicate rows 0


In [ ]:
recipes['dairy-free'] = recipes['tags'].apply(lambda x: 1 if 'dairy-free' in x else 0)
recipes['gluten-free'] = recipes['tags'].apply(lambda x: 1 if 'gluten-free' in x else 0)
recipes['low-carb'] = recipes['tags'].apply(lambda x: 1 if 'low-carb' in x else 0)
recipes['vegan'] = recipes['tags'].apply(lambda x: 1 if 'vegan' in x else 0)
recipes['vegetarian'] = recipes['tags'].apply(lambda x: 1 if 'vegetarian' in x else 0)
recipes['low-calorie'] = recipes['tags'].apply(lambda x: 1 if 'low-calorie' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'easy' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)



In [ ]:

recipes['time'] = recipes['tags'].apply(lambda x: 15 if '15-minutes-or-less' in x else (30 if '30-minutes-or-less' in x else (60 if '60-minutes-or-less' in x else 100)))

In [ ]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic,dairy-free,gluten-free,low-carb,vegan,vegetarian,low-calorie,easy,time
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1,0,0,1,0,1,1,0,100
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",0,0,0,0,0,0,0,0,60
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}",0,0,0,0,0,1,0,1,100
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},0,0,0,0,0,0,0,0,100
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"['slivered almonds', 'cider vinegar', 'sugar',...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"['time-to-make', 'course', 'preparation', 'des...",{'dessert'},0,0,0,0,0,0,0,0,100


In [ ]:
def categorize_meal(search_terms, name):
    if 'breakfast' in search_terms or 'brunch' in search_terms:
        return 'breakfast'
    elif 'lunch' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'lunch'
    elif 'dinner' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'dinner'
    elif 'snack' in search_terms or 'appetizer' in search_terms:
        return 'snack'
    elif 'drink' in search_terms or 'drink' in name or 'Coffee' in name or 'Cocktail' in name:
        return 'drink'
    elif 'side' in search_terms or 'salad' in search_terms:
        return 'side'
    elif 'dessert' in search_terms:
        return 'dessert'
    elif 'bread' in search_terms:
        return 'bread'
    elif 'sauce' in search_terms or 'Sauce' in name:
        return 'sauce'
    else:
        return 'other'

recipes['meal'] = recipes.apply(lambda row: categorize_meal(row['search_terms'], row['name']), axis=1)

In [ ]:
recipes['meal'].value_counts()

meal
dinner       140348
lunch        110243
dessert       82024
side          42875
other         32150
breakfast     30810
snack         25791
bread         13724
sauce          4855
drink          2542
Name: count, dtype: int64

In [ ]:
recipes[recipes['meal']=='dinner'][recipes['diabetic']==1][recipes['vegetarian']==1]

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic,dairy-free,gluten-free,low-carb,vegan,vegetarian,low-calorie,easy,time,meal
1403,297194,Asparagus a La Liz,"This dish results in crispy, lemon accented st...","['asparagus', 'salt and pepper', 'butter', 'le...","[""1/2 lb asparagus, thin preferably "","" s...",1 (149 g),2,"['Trim the ttough ends from the asparagas, rin...","['30-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'low-calorie', 'low-sodium', 'veg...",1,0,0,1,0,1,1,1,30,dinner
1777,346494,Okra &amp; Chickpea Tagine,This spiced dish is full of Moroccan flavour a...,"['okra', 'fresh cilantro', 'extra virgin olive...","[""1 lb okra, stem ends trimmed, cut into ...",1 (227 g),6,['Place a large bowl of ice water next to the ...,"['60-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'dinner', 'vegetarian', 'vegan'}",1,0,0,0,1,1,0,1,60,dinner
1964,274316,Leftover Stuffing Veggi Bites,I came up with this recipe to use left over st...,"['prepared cornbread stuffing', 'peas', 'carro...","[""1 cup prepared cornbread stuffing"",""1/2...",1 (315 g),2,['Boil the potato until cooked. 10 minutes be...,"['30-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'vegetarian', 'appetizer', 'pie',...",1,0,0,0,0,1,0,0,30,dinner
2498,416432,Veggie Crumble Tacos,I sometimes get a craving for good old tacos b...,"['veggie crumbles', 'oil', 'taco seasoning mix...","[""1 lb veggie crumbles, hamburger style ""...",1 (1415 g),1,['Warm veggie crumbles crumbles in sauté pan w...,"['weeknight', '30-minutes-or-less', 'time-to-m...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1,0,0,1,0,1,1,0,30,dinner
2709,252924,Baked Aegean Style Eggplant With Lentils,A lovely use of different flavors and textures...,"['lentils', 'yellow sweet onion', 'japanese eg...","[""2 (15 ounce) cans of your favorite firm l...",1 (422 g),6,['Prepare your ingredients: 1) rinse and drai...,"['60-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'vegetarian', 'baked', 'greek', '...",1,0,0,0,0,1,0,0,60,dinner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492118,95675,Uno Dos Tres Salsa !,This simple recipe makes one of the best salas...,"['diced tomatoes with jalapenos', 'green onion...","[""1 (14 ounce) can petite-cut diced tomatoe...",1 (54 g),8,"['Drain, most of the liquid from tomatoes, dis...","['15-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'vegetarian', 'low-calorie', 'hea...",1,0,1,1,0,1,1,0,15,dinner
493129,125401,Vegetable Galaxy,"A tasty veggie dish, I seen made on TV--it loo...","['zucchini', 'yellow squash', 'eggplant', 'red...","[""2 zucchini, cut into 1/2-inch cubes "",...",1 (508 g),3,"['Heat saute pan until very hot, add olive oil...","['60-minutes-or-less', 'time-to-make', 'course...","{'side', 'diabetic', 'dinner', 'vegetarian'}",1,0,0,0,0,1,0,0,60,dinner
493321,78857,Gala Salsa,This salsa recipe will make enough product for...,"['red pear tomatoes', 'onions', 'jalapenos', '...","[""4 gallons crushed red pear tomatoes"",""10...",1 (402 g),35,"['Mix all ingredients thoroughly.', 'Chill.']","['30-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'low-calorie', 'low-sodium', 'veg...",1,0,0,1,0,1,1,0,30,dinner
494209,156903,Baked (Vegetarian) Chili Spaghetti,"Mmmm, comfort food! I love this topped with c...","['spaghetti', 'fat free spaghetti sauce', 'veg...","[""8 ounces spaghetti, uncooked "",""26 ou...",1 (138 g),6,['Cook pasta according to package. Drain. Whil...,"['60-minutes-or-less', 'time-to-make', 'course...","{'diabetic', 'vegetarian', 'pasta', 'baked', '...",1,0,0,0,0,1,0,1,60,dinner
